In [2]:
import cv2
import numpy as np
import itertools
from scipy.interpolate import LinearNDInterpolator
from PIL import Image
from matplotlib import pyplot as plt

import os, sys
sys.path.append('/home/loitg/workspace/backbone/src/')
from utils import Coord, Line
from khoi import  transform, ppp
from loi import createStandardGrid, multiple_transform, centralize, mls, alignGuidelines, gls2arr,\
drawField, arr1gls, arr2gls, GridMap

sys.path.append('/home/loitg/workspace/test_google/')

from pbprocess import get_document_bounds, FeatureType, readPb, draw_bounds, bound2points

In [3]:
pbin = '/home/loitg/ssd/data/temp3/protobuf_unify_old'
hinhphang1 = '/home/loitg/workspace/dewarp/hinhphang1'
imagestemp = '/home/loitg/workspace/backbone/imagestemp/'
fn = '650000000970875.2dd3731c-184a-4fbe-bc8b-e1333a732e72.636613902329249130.jpg'
image = cv2.imread(os.path.join(hinhphang1, fn))
pb_response = readPb(os.path.join(pbin, fn.replace('jpg', 'pb') ))
word_bounds = get_document_bounds(pb_response, FeatureType.WORD)
boxes = [bound2points(bound) for bound in word_bounds]

mask_word = np.zeros(image.shape[:2], dtype=np.uint8)
direction_map_x = np.zeros(image.shape[:2], dtype=np.uint8)
direction_map_y = np.zeros(image.shape[:2], dtype=np.uint8)
for box in boxes:
    cv2.fillPoly(mask_word, np.expand_dims(np.array(box, dtype=np.int32), axis=0), 1)
    d = Coord(*box[2])-Coord(*box[1])
    cv2.fillPoly(direction_map_x, np.expand_dims(np.array(box, dtype=np.int32), axis=0), d.x)
    cv2.fillPoly(direction_map_y, np.expand_dims(np.array(box, dtype=np.int32), axis=0), d.y)
        
direction_map = np.stack([direction_map_y, direction_map_x], axis=2)
fg_mask = np.ones(image.shape[:2], dtype=np.uint8)

In [4]:
from numpy import random as rd
import random
def createRandomInfos():
    N = rd.randint(1,6) ### fold/curl counts
    infos = []
    for i in range(N):
        center_r_x = rd.rand()
        center_r_y = rd.rand()
        direction = rd.uniform(360)
        #print('%.2f,%.2f--%.2f--%.2f' % (center_r_x, center_r_y, direction, strength))
        method = random.choice(['fold', 'curl'])
        if method == 'fold':
            strength = rd.uniform(0.4,1.3)
        elif method == 'curl':
            strength = rd.uniform(2.0,3.0)
        infos.append((center_r_x, center_r_x, direction, strength, method))
    return infos

In [5]:
infos = [(0.5,0.5,0,1.6,'fold'),\
        (0.5,0.5,90,1.6,'fold')]
# xx0, yy0, xx1, yy1 = distort(image, direction_map, infos)

infos = createRandomInfos()

xx0, yy0 = createStandardGrid(image.shape)
grid_shape = xx0.shape
xx0 = xx0.reshape(-1)
yy0 = yy0.reshape(-1)
xx1, yy1 = multiple_transform(image, xx0, yy0, infos)
xx1, yy1 = centralize(xx1, yy1)
# interpolation

In [5]:
from transform import RotationScale     
t = RotationScale((400,700), 45, 1.0)
rs = t.transform_field(direction_map)

In [10]:
# def hhh(head, p):
#     head[:,:,[0,1]] = head[:,:,[1,0]]
#     a = np.concatenate([head, np.ones(head.shape[:2] + (1,))], axis=2)
#     a = np.expand_dims(a, 3)
#     rs = np.matmul(p, a)
#     rs = rs[:,:,:2,0]
#     rs[:,:,[0,1]] = rs[:,:,[1,0]]
#     return rs

In [11]:
# head = np.moveaxis(np.mgrid[0:image.shape[0]:1, 0:image.shape[1]:1], 0, 2)
# tail = head + direction_map

# h1 = hhh(head, t.perspective)
# t1 = hhh(tail, t.perspective)

# rs = cv2.warpPerspective((t1 - h1), t.perspective, (image.shape[1], image.shape[0]))

image1 = cv2.warpPerspective(image, t.perspective, (image.shape[1], image.shape[0]))

In [12]:
rs = drawField(image1, rs)
cv2.imwrite(os.path.join(imagestemp, '%s_rfield.jpg' % (fn)), rs)

True

In [25]:
# t, r = syn_transform(image, xx1, yy1)
# xx1, yy1 = t.transform_points(xx1, yy1)
mapping_infos = GridMap(np.stack([yy0, xx0], axis=1), np.stack([yy1, xx1], axis=1))

mask_words_infos = mapping_infos.transformArrayScalar(mask_word)

mask_fg_infos = mapping_infos.transformArrayScalar(fg_mask)#.astype(np.uint8)
erode_size = int(image.shape[1]/10)
mask_fg_infos = cv2.erode(mask_fg_infos, np.ones((erode_size,erode_size),np.uint8), iterations = 1)

image_infos = mapping_infos.transformArrayScalar_bilinear(image)

image_infos = image*(mask_fg_infos[:,:,np.newaxis] < 0.5) + image_infos*(mask_fg_infos[:,:,np.newaxis] >= 0.5)

cv2.imwrite(os.path.join(imagestemp, '%s_newimg.jpg' % (fn)), image_infos)
cv2.imwrite(os.path.join(imagestemp, '%s_mask_word.jpg' % (fn)), mask_words_infos*255)
cv2.imwrite(os.path.join(imagestemp, '%s_mask_fg.jpg' % (fn)), mask_fg_infos*255)


True

In [19]:
image.shape[:2]/10

TypeError: unsupported operand type(s) for /: 'tuple' and 'int'

In [11]:
# def createLabelerMaps(image, guidelines):
gls = arr2gls(yy1, xx1, grid_shape)
a_gls = alignGuidelines(gls)
control_points = gls2arr(gls) #to_array2([item for sublist in gls for item in sublist])
dst_points = gls2arr(a_gls) #to_array2([item for sublist in a_gls for item in sublist])
mapping_align = GridMap(control_points, dst_points)

deviation_field = mapping_align.deviationFromShape(image_infos.shape)

image_field = drawField(image_infos, deviation_field)
cv2.imwrite(os.path.join(imagestemp, '%s_field.jpg' % (fn)), image_field)

True

In [10]:
# Reconstruct from field reconstructFromField
# grid = np.moveaxis(np.mgrid[0:image.shape[0]:1, 0:image.shape[1]:1], 0, 2)
# new_grid = (grid + deviation_field).astype(int)
# np.clip(new_grid[:,:,0], 0,image.shape[0]-1, out=new_grid[:,:,0])
# np.clip(new_grid[:,:,1], 0,image.shape[1]-1, out=new_grid[:,:,1])
# new_image = np.ndarray(shape=image.shape, dtype=np.uint8)
# new_image[new_grid[:,:,0].reshape(-1), new_grid[:,:,1].reshape(-1)] = image_infos[grid[:,:,0].reshape(-1), grid[:,:,1].reshape(-1)]
# cv2.imwrite(os.path.join(imagestemp, '%s_transformed.jpg' % (fn)), new_image)

True

In [50]:
pbin = '/home/loitg/ssd/data/temp3/protobuf_unify_old'
hinhphang1 = '/home/loitg/workspace/dewarp/hinhphang1'
imagestemp = '/home/loitg/workspace/backbone/imagestemp/'
    
from scipy import stats

for i, fn in enumerate(os.listdir(hinhphang1)[:1]):
#     if fn not in ['650000000945170.ae71c488-564f-435d-92a6-bbd88725b14d.636613201821177483.jpg']:
#         continue
    print(fn)
    
    image = cv2.imread(os.path.join(hinhphang1, fn))
    direction_map = np.ndarray(shape=(image.shape[0], image.shape[1], 2))
    direction_map[:,:] = [0,-10]

    infos = [(0.5,0.5,0,1.6,'fold'),\
            (0.5,0.5,90,1.6,'fold')]
    xx0, yy0, xx1, yy1 = distort(image, direction_map, infos)
    
    #image_div, direction_map_div = distort([image, direction_map], strength/distort_info=?) ### KHOI
    
    
    

650000000970875.2dd3731c-184a-4fbe-bc8b-e1333a732e72.636613902329249130.jpg


In [9]:
a = cv2.resize(image, None, fx=0.1, fy=0.1)
cv2.imwrite(os.path.join(imagestemp, '%s_small.jpg' % (fn)), a)

True